In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [2]:
! pip install streamlit

In [3]:
! pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.2 MB/s eta 0:00:00


In [6]:
%%writefile htr.py

import streamlit as st
import tensorflow as tf
import numpy as np
import pickle
from PIL import Image
from tensorflow.keras.layers import StringLookup


with open('/content/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)


char_to_num = StringLookup(vocabulary=vocab, mask_token=None)
num_to_chars = StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)


with open('/content/prediction_model.pkl', 'rb') as f:
    prediction_model = pickle.load(f)


max_len = 50

@st.cache(allow_output_mutation=True)
def preprocess_image(image, img_size=(128, 32)):
    image = image.convert('L')
    image = image.resize(img_size, Image.ANTIALIAS)
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=2)
    image = np.transpose(image, (1, 0, 2))
    return image


def predict(image):
    image = preprocess_image(image)
    prediction = prediction_model.predict(np.expand_dims(image, axis=0))
    input_len = np.ones(prediction.shape[0]) * prediction.shape[1]
    decoded = tf.keras.backend.ctc_decode(prediction, input_length=input_len, greedy=True)[0][0][:, :max_len]
    output_text = []
    for res in decoded:
        res = tf.strings.reduce_join(num_to_chars(res)).numpy().decode("utf-8")
        res = res.replace('[UNK]', ' ')
        output_text.append(res)
    return output_text

# Streamlit app
st.title('Handwritten Text Recognition')
uploaded_file = st.file_uploader('Upload an image', type=['png', 'jpg', 'jpeg'])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)
    predictions = predict(image)

    # Font style options
    font_styles = [
    'Arial', 'Book Antiqua', 'Candara', 'Kristen ITC', 'Times New Roman', 'Verdana', 'Tahoma', 'Courier New',
    'Garamond', 'Georgia', 'Lucida Console', 'Palatino Linotype', 'Trebuchet MS', 'Consolas', 'Calibri', 'Cambria',
    'Segoe UI', 'Helvetica', 'Futura', 'Franklin Gothic', 'Gill Sans', 'Optima', 'Baskerville', 'Bodoni',
    'Didot', 'Rockwell', 'Perpetua', 'Goudy Old Style', 'Playfair Display', 'Lora', 'Merriweather', 'Roboto',
    'Open Sans', 'Lato', 'Montserrat', 'Source Sans Pro', 'Raleway', 'Oswald', 'Ubuntu', 'Droid Sans',
    'Droid Serif', 'PT Sans', 'PT Serif', 'Bitter', 'Crimson Text', 'Slabo 27px', 'Dosis', 'Archivo Black',
    'Righteous', 'Lobster', 'Shadows Into Light', 'Pacifico', 'Satisfy', 'Handlee', 'Neucha', 'Permanent Marker'
]
    default_font = 'Arial'
    selected_font = st.selectbox('Select Font Style', font_styles, index=font_styles.index(default_font))

    st.write('Predictions:')
    for pred in predictions:
        st.write(f'<span style="font-family:{selected_font}; font-size:24px;">{pred}</span>', unsafe_allow_html=True)

Overwriting htr.py


In [7]:
!streamlit run htr.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.7.217:8501

npx: installed 22 in 3.224s
your url is: https://four-cameras-wear.loca.lt
2024-05-17 05:58:11.380103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 05:58:11.380204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 05:58:11.384393: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 05:58:11.404035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions